In [1]:
from transformers import pipeline
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

print(torch.cuda.device_count())
#os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

#モデル読み込み
model_name = "meta-llama/Llama-2-7b-chat-hf"

model_name = "tokyotech-llm/Swallow-MS-7b-v0.1"
model_name = "cyberagent/open-calm-7b"
model_name="../model/------llm-models-hf-0503llama-_inst_gousei_lr_5e-6"
model_name="../model/------llm-models-hf-2jaA_inst_gousei_lr_1e-5/checkpoint-47"
model_name="../model/------llm-models-hf-2jaA_inst_gousei_lr_5e-6"
model_name="Local-Novel-LLM-project/Vecteus-v1"
#model_name="nvidia/Llama3-ChatQA-1.5-8B"
#model_name="../model/------llm-models-hf-1code_inst_gousei_lr_5e-6"
model_name="../../llm/models/hf/1code"
model_name="../../llm/models/hf/2ja1_part"

#model_name="../X_merge/mergoo_llama_test"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )
#層の表示
#for name, param in model.named_parameters():
#    print(name)

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


8


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, 

In [2]:
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()



In [3]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=300, 
              repetition_penalty=1.2,
              temperature=0.6,
              )


In [4]:
questions=[
"こんにちは",
"元気ですか?",
"いい天気ですね",
"明日の天気は?",
"計算してください 1+1はいくつですか｡",
"フィボナッチ数列を生成するpythonのコードを作成してください",
"ドラえもんの友達はだれですか?",
"レイリー散乱とはなんですか",
"hello!",
"今の天気は晴れです｡この結果をjsonで出力してください｡ ",
]
for question in questions:
    question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    inp=f"{question_template}{question}<SEP>応答<SEP>"
    print("--------------------")
    print(question)
    print(pipe(inp)[0]["generated_text"][len(inp):])

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


--------------------
こんにちは
 ありがとうございます今日もお世話になりましたね!
2018年6月3日(水)19:57:40
本日は、私が所属している「日本サッカー協会」の理事会で行われました。
私は、この理事会には参加していませんが、今回は、私が会長として務めさせていただきました。
私は、この理事会では、毎週水曜日に開催される「第九土曜日の試合」に出場しました。
私は、この理事会の主催者である、株式会社代表取締役社長の長谷川氏から、「第九土曜日の試合」についてお話を聞きました。
私は、この理事会の主催者であり、取締役社長の長谷川氏から、第九土曜日の試合についてお話を聞きました。
私は、この理事会の主催者であり、取締役社長の長谷川氏から、第九土曜日の試合について話を聞きました。
私は、この理事会の主催者であり、取締役社長の長谷川氏から、第九土曜日の試合について話を聞きました。
私は、この理事会の主催者であり、取締役社長の長谷川氏から、第九土曜日の試合について話を聞きました。
私は、この理事会の主催者であり、取締役社長の長谷川氏から、第九土曜日の試合について話を聞き
--------------------
元気ですか?
 元気な人がいるから頑張れよ!
このように書いています。
「自分で考える」という言葉がありますね。
私もそう思います。
でも、その場合は、
「自分で決めた結果が出ている」
と言うのが正しいと思っています。
それは、
「自分で決めたことだから」
ということです。
私自身、
「自分で決めたことを実行していれば、それでいい」
と思うようになりました。
しかしながら、
「自分で決めたことだから」
というのは、
決して間違ったものではありません。
私は、「自分で決めたことだから」
ということを言っていません。
ただ、
「自分で決めたことだから」
ということだけは言えません。
それどころか、
「自分で決めたことだから」
と言うことすらありません。
もし、
「自分で決めたことだから」
と言うことが出来れば、
「自分で決めたことだから」
と思えるようになるはずです。
では、
「自分で決めたことだから」
と言う事になるでしょう。
それを実現するためには、
「自分で決めたことだから」
と言う事になります。
そして、
「自分で決めたことだから」
と言